In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve
from ast import literal_eval

import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV, LinearRegression

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, overview, roc_results, model
from lob_data_utils.svm_calculation import lob_svm

sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

/home/vevurka/.conda/envs/py36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
data_length = 10000 # TODO: not used
stock = '9064'

In [3]:
df, df_test = lob.load_prepared_data(
    stock, data_dir='../queue_imbalance/data/prepared', cv=False, include_test=True, length=None)
print(len(df), len(df_test))

14141 3535


In [4]:
def prepare_df(df):
    df.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    df.drop(columns=['bid', 'ask'], inplace=True)
    df.index = df['datetime']
    df = df.sort_index()
    df['prev_queue_imbalance'] = [None] + df['queue_imbalance'].iloc[0:len(df)-1].values.tolist()
    df.dropna(inplace=True)
    return df
df = prepare_df(df)
df_test = prepare_df(df_test)
df.head()

,datetime,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance
datetime,,,,,,,,,
2013-09-16 12:24:00,2013-09-16 12:24:00,77.08,77.10,77.090,53636.0,100.0,1.0,-0.996278,0.261169
2013-09-16 12:25:00,2013-09-16 12:25:00,77.09,77.12,77.105,26303.0,31203.0,1.0,0.085208,-0.996278
2013-09-16 12:26:00,2013-09-16 12:26:00,77.13,77.16,77.145,3400.0,25554.0,1.0,0.765145,0.085208
2013-09-16 12:27:00,2013-09-16 12:27:00,77.16,77.19,77.175,6735.0,30903.0,1.0,0.642117,0.765145
2013-09-16 12:28:00,2013-09-16 12:28:00,77.17,77.19,77.180,4610.0,17290.0,1.0,0.578995,0.642117


## Let's try to fit Logistic Regression

In [5]:
df_result = pd.DataFrame()
features = ['queue_imbalance', 'prev_queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

features = ['queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc
0,"[0.5894279507603185, 0.5661764705882353, 0.572...","queue_imbalance, prev_queue_imbalance","[0.11527276038978784, 0.08072053370711585, 0.0...","[0.11598674111000024, 0.08109983227920431, 0.0...",logistic,"[0.5606060606060606, 0.5414908579465542, 0.552...","[0.6213740458015267, 0.5932203389830508, 0.593...","[0.5575124197261602, 0.5403208613154248, 0.548...","[0.5652487382840664, 0.5905429071803853, 0.563...","[0.05910645743754639, 0.08680813709261193, 0.0...","[0.05956070185810295, 0.08833523169395377, 0.0...","[0.5347885402455662, 0.5452781371280725, 0.543...","[0.599388379204893, 0.6440030557677616, 0.5858...","[0.5294881674154417, 0.5432545017141742, 0.538..."
1,"[0.5818181818181819, 0.539291217257319, 0.5659...",queue_imbalance,"[0.1029532279114942, 0.06916543109379691, 0.08...","[0.10348668127725809, 0.06916543109379694, 0.0...",logistic,"[0.5555555555555556, 0.539291217257319, 0.5472...","[0.6106870229007634, 0.539291217257319, 0.5858...","[0.5513752574821278, 0.5345827155468985, 0.543...","[0.5728643216080402, 0.5756698044895004, 0.567...","[0.07143368962330265, 0.08570519665880072, 0.0...","[0.07207060421073702, 0.08625114786058516, 0.0...","[0.5399188092016238, 0.547143840330351, 0.5455...","[0.6100917431192661, 0.6073338426279603, 0.590...","[0.5356322423995696, 0.5427628769047811, 0.541..."


In [6]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]
coef0s = [0.1, 1, 10, 100]

for g in gammas:
    for c in cs:
        for coef in coef0s:
            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance', 'prev_queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)

            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.5894279507603185, 0.5661764705882353, 0.572...","queue_imbalance, prev_queue_imbalance","[0.11527276038978784, 0.08072053370711585, 0.0...","[0.11598674111000024, 0.08109983227920431, 0.0...",logistic,"[0.5606060606060606, 0.5414908579465542, 0.552...","[0.6213740458015267, 0.5932203389830508, 0.593...","[0.5575124197261602, 0.5403208613154248, 0.548...","[0.5652487382840664, 0.5905429071803853, 0.563...","[0.05910645743754639, 0.08680813709261193, 0.0...","[0.05956070185810295, 0.08833523169395377, 0.0...","[0.5347885402455662, 0.5452781371280725, 0.543...","[0.599388379204893, 0.6440030557677616, 0.5858...","[0.5294881674154417, 0.5432545017141742, 0.538...",NaN,NaN,NaN
1,"[0.5818181818181819, 0.539291217257319, 0.5659...",queue_imbalance,"[0.1029532279114942, 0.06916543109379691, 0.08...","[0.10348668127725809, 0.06916543109379694, 0.0...",logistic,"[0.5555555555555556, 0.539291217257319, 0.5472...","[0.6106870229007634, 0.539291217257319, 0.5858...","[0.5513752574821278, 0.5345827155468985, 0.543...","[0.5728643216080402, 0.5756698044895004, 0.567...","[0.07143368962330265, 0.08570519665880072, 0.0...","[0.07207060421073702, 0.08625114786058516, 0.0...","[0.5399188092016238, 0.547143840330351, 0.5455...","[0.6100917431192661, 0.6073338426279603, 0.590...","[0.5356322423995696, 0.5427628769047811, 0.541...",NaN,NaN,NaN
2,"[0.6752577319587628, 0.5359073359073359, 0.550...","queue_imbalance, prev_queue_imbalance","[0.0, 0.06453988592082771, 0.07972148134504242...","[0.0, 0.06454058955903784, 0.0797331787606132,...",svm_sigmoid,"[0.5097276264591439, 0.5371517027863777, 0.546...","[1.0, 0.5346687211093991, 0.5552147239263804, ...","[0.5, 0.5322714674729386, 0.5398506479031586, ...","[0.6745745229499742, 0.5757795503988398, 0.555...","[0.0, 0.08731861865161483, 0.0809991126971582,...","[0.0, 0.08784403766711678, 0.08104386912157188...","[0.5089494163424124, 0.5479641131815045, 0.546...","[1.0, 0.6065699006875478, 0.5643513789581205, ...","[0.5, 0.543570438051942, 0.5404782725048904, 0...",0.1,0.1,0.1
3,"[0.6752577319587628, 0.539291217257319, 0.5432...",queue_imbalance,"[0.0, 0.06916543109379691, 0.07054975011358466...","[0.0, 0.06916543109379694, 0.07055051963149776...",svm_sigmoid,"[0.5097276264591439, 0.539291217257319, 0.5419...","[1.0, 0.539291217257319, 0.5444785276073619, 0...","[0.5, 0.5345827155468985, 0.5352724391591313, ...","[0.6745745229499742, 0.5756698044895004, 0.554...","[0.0, 0.08570519665880072, 0.08155605084569773...","[0.0, 0.08625114786058516, 0.08159434086878273...","[0.5089494163424124, 0.547143840330351, 0.5468...","[1.0, 0.6073338426279603, 0.5633299284984679, ...","[0.5, 0.5427628769047811, 0.5407582694680004, ...",0.1,0.1,0.1
4,"[0.6752577319587628, 0.671147880041365, 0.6580...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.043048604527538226, 0.04893909248...","[0.0, 0.0, 0.06464761371322367, 0.050470505833...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.519...","[1.0, 1.0, 0.897239263803681, 0.66666666666666...","[0.5, 0.5, 0.5212894581261691, 0.5241979336595...","[0.6745745229499742, 0.6749162155194638, 0.660...","[0.0, 0.0, 0.025019193461502387, 0.08197257290...","[0.0, 0.0, 0.042763953058764986, 0.08505879443...","[0.5089494163424124, 0.5093385214007782, 0.514...","[1.0, 1.0, 0.9203268641470889, 0.6808429118773...","[0.5, 0.5, 0.5123510968073346, 0.5408167128556...",0.1,1.0,0.1
5,"[0.6752577319587628, 0.671147880041365, 0.6603...",queue_imbalance,"[0.0, 0.0, 0.0460632699262048, 0.0563468146194...","[0.0, 0.0, 0.07032121530780112, 0.058283430423...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.520...","[1.0, 1.0, 0.9033742331288344, 0.6755952380952...","[0.5, 0.5, 0.5227771639577821, 0.5278465586887...","[0.6745745229499742, 0.6749162155194638, 0.662...","[0.0, 0.0, 0.028577179000694453, 0.08308367069...","[0.0

In [7]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]

for g in gammas:
    for c in cs:
        clf = SVC(kernel='rbf', gamma=g, C=c)
        features = ['queue_imbalance', 'prev_queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)

        clf = SVC(kernel='sigmoid', gamma=g, C=c)
        features = ['queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.5894279507603185, 0.5661764705882353, 0.572...","queue_imbalance, prev_queue_imbalance","[0.11527276038978784, 0.08072053370711585, 0.0...","[0.11598674111000024, 0.08109983227920431, 0.0...",logistic,"[0.5606060606060606, 0.5414908579465542, 0.552...","[0.6213740458015267, 0.5932203389830508, 0.593...","[0.5575124197261602, 0.5403208613154248, 0.548...","[0.5652487382840664, 0.5905429071803853, 0.563...","[0.05910645743754639, 0.08680813709261193, 0.0...","[0.05956070185810295, 0.08833523169395377, 0.0...","[0.5347885402455662, 0.5452781371280725, 0.543...","[0.599388379204893, 0.6440030557677616, 0.5858...","[0.5294881674154417, 0.5432545017141742, 0.538...",NaN,NaN,NaN
1,"[0.5818181818181819, 0.539291217257319, 0.5659...",queue_imbalance,"[0.1029532279114942, 0.06916543109379691, 0.08...","[0.10348668127725809, 0.06916543109379694, 0.0...",logistic,"[0.5555555555555556, 0.539291217257319, 0.5472...","[0.6106870229007634, 0.539291217257319, 0.5858...","[0.5513752574821278, 0.5345827155468985, 0.543...","[0.5728643216080402, 0.5756698044895004, 0.567...","[0.07143368962330265, 0.08570519665880072, 0.0...","[0.07207060421073702, 0.08625114786058516, 0.0...","[0.5399188092016238, 0.547143840330351, 0.5455...","[0.6100917431192661, 0.6073338426279603, 0.590...","[0.5356322423995696, 0.5427628769047811, 0.541...",NaN,NaN,NaN
2,"[0.6752577319587628, 0.5359073359073359, 0.550...","queue_imbalance, prev_queue_imbalance","[0.0, 0.06453988592082771, 0.07972148134504242...","[0.0, 0.06454058955903784, 0.0797331787606132,...",svm_sigmoid,"[0.5097276264591439, 0.5371517027863777, 0.546...","[1.0, 0.5346687211093991, 0.5552147239263804, ...","[0.5, 0.5322714674729386, 0.5398506479031586, ...","[0.6745745229499742, 0.5757795503988398, 0.555...","[0.0, 0.08731861865161483, 0.0809991126971582,...","[0.0, 0.08784403766711678, 0.08104386912157188...","[0.5089494163424124, 0.5479641131815045, 0.546...","[1.0, 0.6065699006875478, 0.5643513789581205, ...","[0.5, 0.543570438051942, 0.5404782725048904, 0...",0.1,0.1,0.1
3,"[0.6752577319587628, 0.539291217257319, 0.5432...",queue_imbalance,"[0.0, 0.06916543109379691, 0.07054975011358466...","[0.0, 0.06916543109379694, 0.07055051963149776...",svm_sigmoid,"[0.5097276264591439, 0.539291217257319, 0.5419...","[1.0, 0.539291217257319, 0.5444785276073619, 0...","[0.5, 0.5345827155468985, 0.5352724391591313, ...","[0.6745745229499742, 0.5756698044895004, 0.554...","[0.0, 0.08570519665880072, 0.08155605084569773...","[0.0, 0.08625114786058516, 0.08159434086878273...","[0.5089494163424124, 0.547143840330351, 0.5468...","[1.0, 0.6073338426279603, 0.5633299284984679, ...","[0.5, 0.5427628769047811, 0.5407582694680004, ...",0.1,0.1,0.1
4,"[0.6752577319587628, 0.671147880041365, 0.6580...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.043048604527538226, 0.04893909248...","[0.0, 0.0, 0.06464761371322367, 0.050470505833...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.519...","[1.0, 1.0, 0.897239263803681, 0.66666666666666...","[0.5, 0.5, 0.5212894581261691, 0.5241979336595...","[0.6745745229499742, 0.6749162155194638, 0.660...","[0.0, 0.0, 0.025019193461502387, 0.08197257290...","[0.0, 0.0, 0.042763953058764986, 0.08505879443...","[0.5089494163424124, 0.5093385214007782, 0.514...","[1.0, 1.0, 0.9203268641470889, 0.6808429118773...","[0.5, 0.5, 0.5123510968073346, 0.5408167128556...",0.1,1.0,0.1
5,"[0.6752577319587628, 0.671147880041365, 0.6603...",queue_imbalance,"[0.0, 0.0, 0.0460632699262048, 0.0563468146194...","[0.0, 0.0, 0.07032121530780112, 0.058283430423...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.520...","[1.0, 1.0, 0.9033742331288344, 0.6755952380952...","[0.5, 0.5, 0.5227771639577821, 0.5278465586887...","[0.6745745229499742, 0.6749162155194638, 0.662...","[0.0, 0.0, 0.028577179000694453, 0.08308367069...","[0.0

In [8]:
cs = [0.1, 1, 10, 100]

for c in cs:
    clf = SVC(kernel='linear', C=c)
    features = ['queue_imbalance', 'prev_queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)

    clf = SVC(kernel='sigmoid', gamma=g, C=c)
    features = ['queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,"[0.5894279507603185, 0.5661764705882353, 0.572...","queue_imbalance, prev_queue_imbalance","[0.11527276038978784, 0.08072053370711585, 0.0...","[0.11598674111000024, 0.08109983227920431, 0.0...",logistic,"[0.5606060606060606, 0.5414908579465542, 0.552...","[0.6213740458015267, 0.5932203389830508, 0.593...","[0.5575124197261602, 0.5403208613154248, 0.548...","[0.5652487382840664, 0.5905429071803853, 0.563...","[0.05910645743754639, 0.08680813709261193, 0.0...","[0.05956070185810295, 0.08833523169395377, 0.0...","[0.5347885402455662, 0.5452781371280725, 0.543...","[0.599388379204893, 0.6440030557677616, 0.5858...","[0.5294881674154417, 0.5432545017141742, 0.538...",NaN,NaN,NaN
1,"[0.5818181818181819, 0.539291217257319, 0.5659...",queue_imbalance,"[0.1029532279114942, 0.06916543109379691, 0.08...","[0.10348668127725809, 0.06916543109379694, 0.0...",logistic,"[0.5555555555555556, 0.539291217257319, 0.5472...","[0.6106870229007634, 0.539291217257319, 0.5858...","[0.5513752574821278, 0.5345827155468985, 0.543...","[0.5728643216080402, 0.5756698044895004, 0.567...","[0.07143368962330265, 0.08570519665880072, 0.0...","[0.07207060421073702, 0.08625114786058516, 0.0...","[0.5399188092016238, 0.547143840330351, 0.5455...","[0.6100917431192661, 0.6073338426279603, 0.590...","[0.5356322423995696, 0.5427628769047811, 0.541...",NaN,NaN,NaN
2,"[0.6752577319587628, 0.5359073359073359, 0.550...","queue_imbalance, prev_queue_imbalance","[0.0, 0.06453988592082771, 0.07972148134504242...","[0.0, 0.06454058955903784, 0.0797331787606132,...",svm_sigmoid,"[0.5097276264591439, 0.5371517027863777, 0.546...","[1.0, 0.5346687211093991, 0.5552147239263804, ...","[0.5, 0.5322714674729386, 0.5398506479031586, ...","[0.6745745229499742, 0.5757795503988398, 0.555...","[0.0, 0.08731861865161483, 0.0809991126971582,...","[0.0, 0.08784403766711678, 0.08104386912157188...","[0.5089494163424124, 0.5479641131815045, 0.546...","[1.0, 0.6065699006875478, 0.5643513789581205, ...","[0.5, 0.543570438051942, 0.5404782725048904, 0...",0.1,0.1,0.1
3,"[0.6752577319587628, 0.539291217257319, 0.5432...",queue_imbalance,"[0.0, 0.06916543109379691, 0.07054975011358466...","[0.0, 0.06916543109379694, 0.07055051963149776...",svm_sigmoid,"[0.5097276264591439, 0.539291217257319, 0.5419...","[1.0, 0.539291217257319, 0.5444785276073619, 0...","[0.5, 0.5345827155468985, 0.5352724391591313, ...","[0.6745745229499742, 0.5756698044895004, 0.554...","[0.0, 0.08570519665880072, 0.08155605084569773...","[0.0, 0.08625114786058516, 0.08159434086878273...","[0.5089494163424124, 0.547143840330351, 0.5468...","[1.0, 0.6073338426279603, 0.5633299284984679, ...","[0.5, 0.5427628769047811, 0.5407582694680004, ...",0.1,0.1,0.1
4,"[0.6752577319587628, 0.671147880041365, 0.6580...","queue_imbalance, prev_queue_imbalance","[0.0, 0.0, 0.043048604527538226, 0.04893909248...","[0.0, 0.0, 0.06464761371322367, 0.050470505833...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.519...","[1.0, 1.0, 0.897239263803681, 0.66666666666666...","[0.5, 0.5, 0.5212894581261691, 0.5241979336595...","[0.6745745229499742, 0.6749162155194638, 0.660...","[0.0, 0.0, 0.025019193461502387, 0.08197257290...","[0.0, 0.0, 0.042763953058764986, 0.08505879443...","[0.5089494163424124, 0.5093385214007782, 0.514...","[1.0, 1.0, 0.9203268641470889, 0.6808429118773...","[0.5, 0.5, 0.5123510968073346, 0.5408167128556...",0.1,1.0,0.1
5,"[0.6752577319587628, 0.671147880041365, 0.6603...",queue_imbalance,"[0.0, 0.0, 0.0460632699262048, 0.0563468146194...","[0.0, 0.0, 0.07032121530780112, 0.058283430423...",svm_sigmoid,"[0.5097276264591439, 0.5050583657587548, 0.520...","[1.0, 1.0, 0.9033742331288344, 0.6755952380952...","[0.5, 0.5, 0.5227771639577821, 0.5278465586887...","[0.6745745229499742, 0.6749162155194638, 0.662...","[0.0, 0.0, 0.028577179000694453, 0.08308367069...","[0.0

In [9]:
df_result.to_csv('res_{}_prev_queue_imbalance.csv'.format(stock))